### Логистическая регрессия

### 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. 
Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [1]:
import json
import bz2

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import datetime
import time

In [2]:
features = pd.read_csv('./data/features.csv', index_col='match_id')
y = features['radiant_win']
features = features.drop(['duration', 
                          'tower_status_radiant', 'tower_status_dire', 
                          'barracks_status_radiant', 'barracks_status_dire',
                          'radiant_win'], axis=1)
features = features.fillna(0)
s = features.count()
s[s != 97230]

Series([], dtype: int64)

In [3]:
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [4]:
clf = LogisticRegression(penalty='l2', random_state=241)
cv = KFold(n_splits=5, shuffle=True, random_state=241)

grid = {'C': np.arange(.5, 10.5, .5)}
gs1 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [5]:
start_time1 = datetime.datetime.now()
gs1.fit(features, y)
end_time1 = datetime.datetime.now()
print(end_time1 - start_time1)

C:\Users\aaa\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0:00:32.301526


In [6]:
print(f"{gs1.best_estimator_}\n")
for k, v in gs1.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=241, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Key: mean_fit_time, value: [0.81253533 0.86110835 0.89488754 0.92467141 0.90228343 0.94545922
 1.11176505 1.08497939 0.922473   0.89009194 0.85691051 1.04060411
 1.20750933 1.06658969 0.97584186 0.80274196 0.79674363 0.79754357
 0.81013641 0.76376209]

Key: mean_score_time, value: [0.03038354 0.03378062 0.03198261 0.03278217 0.0363802  0.04597335
 0.05576816 0.04117761 0.03717866 0.04277577 0.03477998 0.04597573
 0.04777427 0.04717321 0.03418155 0.03318033 0.0337821  0.03318157
 0.03238196 0.03078256]

Key: mean_test_score, value: [0.51345564 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564
 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564
 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564 0.5134556

### Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. 
Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [7]:
features_int = features.drop(['lobby_type', 
                              'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                              'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'],
                            axis=1)
clf = LogisticRegression(penalty='l2', random_state=241)
cv = KFold(n_splits=5, shuffle=True, random_state=241)
grid = {'C': np.arange(.5, 10.5, .5)}
gs2 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [8]:
start_time2 = datetime.datetime.now()
gs2.fit(features, y)
end_time2 = datetime.datetime.now()
print(end_time2 - start_time2)

C:\Users\aaa\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\aaa\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0:00:27.342963


In [9]:
print(f"{gs2.best_estimator_}\n")
for k, v in gs2.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=241, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Key: mean_fit_time, value: [0.79934316 0.84971418 0.83012547 0.91307859 0.81973209 0.82472811
 0.82412877 0.81373472 0.80214233 0.84851513 0.89928584 0.85731068
 0.85391278 0.89169116 0.84171848 0.81533351 0.82574239 0.76915994
 0.7785531  0.74397473]

Key: mean_score_time, value: [0.02978363 0.0393775  0.03098269 0.02698436 0.02758493 0.02878509
 0.0259861  0.02958369 0.03298097 0.03118248 0.02878327 0.03338003
 0.0319829  0.02938251 0.03038492 0.02758565 0.02878423 0.02778435
 0.0279851  0.02418685]

Key: mean_test_score, value: [0.51850252 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252
 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252
 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252 0.5185025

### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. 
Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [10]:
heroes = pd.read_csv('./data/dictionaries/heroes.csv')
print(f"{len(heroes['name'].unique())}")  # Кол-во героев
heroes.head()

112


,id,localized_name,name
0,1,Anti-Mage,antimage
1,2,Axe,axe
2,3,Bane,bane
3,4,Bloodseeker,bloodseeker
4,5,Crystal Maiden,crystal_maiden


### 4. Какое получилось качество при добавлении "мешка слов" по героям? 
Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

In [11]:
features = pd.read_csv('./data/features.csv', index_col='match_id')
y = features['radiant_win']
features = features.drop(['duration', 
                          'tower_status_radiant', 'tower_status_dire', 
                          'barracks_status_radiant', 'barracks_status_dire',
                          'radiant_win'
                         ], axis=1)
features = features.fillna(0)

# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

C:\Users\aaa\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
C:\Users\aaa\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [12]:
bag = np.concatenate((features.to_numpy(), X_pick), axis=1)
bag.shape

(97230, 214)

In [13]:
clf = LogisticRegression(penalty='l2', random_state=241)
cv = KFold(n_splits=5, shuffle=True, random_state=241)
grid = {'C': np.arange(.5, 10.5, .5)}
gs3 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [14]:
start_time3 = datetime.datetime.now()
gs3.fit(bag, y)
end_time3 = datetime.datetime.now()
print(end_time3 - start_time3)

C:\Users\aaa\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\aaa\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0:00:36.921788


In [15]:
print(f"{gs3.best_estimator_}\n")
for k, v in gs3.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=241, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Key: mean_fit_time, value: [1.2434895  1.17912664 1.16093431 1.17932582 1.14734306 1.14274478
 1.14574375 1.19191766 1.24408836 1.41199231 1.17932639 1.17204781
 1.18160353 1.18635788 1.18152561 1.1743289  1.18851986 1.18012543
 1.19311872 1.10796618]

Key: mean_score_time, value: [0.01579027 0.0151916  0.01439242 0.01679144 0.01619124 0.01499143
 0.01679087 0.02438607 0.01919007 0.01838822 0.01499128 0.01419163
 0.01458583 0.01599002 0.01499038 0.01539183 0.01439247 0.01399131
 0.01519117 0.01339293]

Key: mean_test_score, value: [0.51850252 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252
 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252
 0.51850252 0.51850252 0.51850252 0.51850252 0.51850252 0.5185025